In [1]:

import sys
import pandas as pd
import os
import numpy as np

from dotenv import load_dotenv
from functools import wraps
from requests.exceptions import ReadTimeout
from binance.client import Client
from datetime import datetime, timedelta
sys.path.append('/Users/goldenhello/Desktop/Quant/Matic_Strategy')

/var/folders/tz/42t89gc96wlc7x7hvqz5gqnc0000gn/T/ipykernel_26575/1612467463.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/Users/goldenhello/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
load_dotenv()
binance_api_key= os.getenv('BINANCE_API_KEY')
binance_secret_key= os.getenv('BINANCE_API_SECRET')
client = Client(binance_api_key, binance_secret_key)

In [3]:
# Convert start_date to new format
start_date_formatted = '01 March, 2024'

# Convert end_date_plus1 to new format
end_date_plus1_formatted = '30 April, 2024'

# Print the formatted dates
print("Start Date:", start_date_formatted)
print("End Date", end_date_plus1_formatted)

Start Date: 01 March, 2024
End Date 30 April, 2024


In [4]:
RSI= 300
n_back = 10
n_back_full = max(n_back, RSI)
x_forward = 10

In [5]:
klines = client.get_historical_klines("MATICUSDT", Client.KLINE_INTERVAL_1MINUTE, start_date_formatted, end_date_plus1_formatted)
column_names = ["Open time", "Open", "High", "Low", "Close", "Volume", "Close time", "Quote asset volume", "Number of trades", "Taker buy base asset volume", "Taker buy quote asset volume", "Ignore"]

# Run Once Start Here

In [ ]:
df = pd.DataFrame(klines, columns=column_names)

In [6]:
df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')

# Convert 'Open' and 'Close' columns to numeric
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['Close'] = pd.to_numeric(df['Close'], errors='coerce')

# Calculate Gain/Loss
df['Gain/Loss'] = df['Close'] - df['Open']

# Calculate Percentage Change
df['Percentage Change'] = (df['Gain/Loss'] / df['Open']) * 100

sign_changes = np.sign(df['Percentage Change']).diff().ne(0)
df['Conditional Cumulative Sum'] = df.groupby(sign_changes.cumsum())['Percentage Change'].cumsum()

# Calculate the 300-period moving average of the 'Close' prices
# Since this example has fewer than 300 periods, replace '300' with a smaller number like '3' to see the calculation
df['300 Period MA'] = df['Close'].rolling(window=300, min_periods=1).mean()
df['60 Period MA'] = df['Close'].rolling(window=60, min_periods=1).mean()
df['14 Period MA'] = df['Close'].rolling(window=14, min_periods=1).mean()
df['Distance_300_MA'] = df['Open']-df['300 Period MA']

# Previous values are lower in the dataset due to descending order
for i in range(1, n_back+1):
    df[f'Close_prev_{i}'] = df['Close'].shift(i)

# Future values are higher in the dataset due to descending order
for i in range(1, x_forward+1):
    df[f'Close_future_{i}'] = df['Close'].shift(-i)


In [7]:
def calculate_rsi(df, periods=14):
    delta = df['Open'].diff()
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)

    # Calculate the exponential moving average
    avg_gain = gain.ewm(com=periods-1, min_periods=periods).mean()
    avg_loss = loss.ewm(com=periods-1, min_periods=periods).mean()

    # Calculate RS and RSI
    # Add a small number to avg_loss to avoid division by zero
    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))

    return rsi

# Calculate RSI
df['RSI_300'] = calculate_rsi(df, periods=300)

In [8]:
# Convert all relevant columns to numeric, missing values will be NaN
df = df.apply(pd.to_numeric, errors='coerce')

# Define the function to process the highest future close difference
def process_highest_difference(row):
    close_value = row['Close']
    # Extract all future close values from the row, ignoring NaN values
    future_values = [row[col] for col in df if col.startswith('Close_future_') and pd.notna(row[col])]
    if not future_values or pd.isna(close_value):  # Check if future_values is empty or close_value is NaN
        return 0  # Return 0 if there are no valid future values or if close is NaN

    # Find the maximum future value
    max_future_value = max(future_values)
    # Calculate the percentage difference and round to three decimal places
    percentage_difference = round((max_future_value - close_value) / close_value, 3)
    # Handle negative values
    if percentage_difference < 0:
        return 0
    # Extract the last decimal digit and cap it at 7
    last_digit = int((percentage_difference * 1000) % 10)
    return min(last_digit, 7)

# Apply the function to each row and create a new column
df['Processed Difference'] = df.apply(process_highest_difference, axis=1)

df.to_json('data_futures.json')

df = df.drop(columns=[
       'Close_future_1', 'Close_future_2', 'Close_future_3', 'Close_future_4',
       'Close_future_5', 'Close_future_6', 'Close_future_7', 'Close_future_8',
       'Close_future_9', 'Close_future_10','Close time','Ignore'])

df = df.iloc[n_back_full:-x_forward]

In [9]:
df.to_json('data_1.json')

In [10]:
df.columns


Index(['Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
       'Taker buy quote asset volume', 'Gain/Loss', 'Percentage Change',
       'Conditional Cumulative Sum', '300 Period MA', '60 Period MA',
       '14 Period MA', 'Distance_300_MA', 'Close_prev_1', 'Close_prev_2',
       'Close_prev_3', 'Close_prev_4', 'Close_prev_5', 'Close_prev_6',
       'Close_prev_7', 'Close_prev_8', 'Close_prev_9', 'Close_prev_10',
       'RSI_300', 'Processed Difference'],
      dtype='object')

In [11]:
with pd.option_context('display.max_columns', None):  # None means unlimited
    print(df)

                 Open time    Open    High     Low   Close    Volume  \
300    1709269200000000000  1.0216  1.0227  1.0215  1.0222   24700.5   
301    1709269260000000000  1.0222  1.0223  1.0213  1.0214   14304.7   
302    1709269320000000000  1.0214  1.0215  1.0192  1.0199   88450.0   
303    1709269380000000000  1.0200  1.0209  1.0200  1.0202   49417.1   
304    1709269440000000000  1.0202  1.0221  1.0199  1.0221   82584.6   
...                    ...     ...     ...     ...     ...       ...   
86386  1714434360000000000  0.7081  0.7081  0.7058  0.7060  181084.5   
86387  1714434420000000000  0.7059  0.7061  0.7056  0.7057   40075.8   
86388  1714434480000000000  0.7057  0.7057  0.7049  0.7054  112907.3   
86389  1714434540000000000  0.7053  0.7055  0.7052  0.7055   20243.2   
86390  1714434600000000000  0.7055  0.7060  0.7051  0.7056   12743.3   

       Quote asset volume  Number of trades  Taker buy base asset volume  \
300           25248.98119               101                